参考文章： https://mp.weixin.qq.com/s/Fi13jaEkM5EGjmS7Mm_Bjw

再附带本人曾写过的文章 ：https://blog.csdn.net/sun91019718/article/details/101314545

# 混淆矩阵
- 分类算法常用评价方法

## 基本定义

- TP:真实值是1，预测值也是1;True of Positive
- FN:真实值是1，预测值是0;False of Negative
- TN:真实值是0，预测值也是0;True of Positive
- FP:真实值是0，预测值是1;False of Positive

- TP+FN：真实1总数；TP+FP:预测1总数

## 三个重要指标

### 精准率
precision=TP/(TP+FP)  也叫 查准率

引用饼干Japson的话：为什么管它叫精准率呢？在有偏的数据中，我们通常更关注值为1的特征，比如“患病”，比如“有风险”。
在100次结果为患病的预测，平均有40次预测是对的。即精准率为我们关注的那个事件，预测的有多准。

### 召回率
recall=TP/(TP+FN)  即 TPR

也叫：真正率或灵敏度或召回率或查全率或真正率或功效。找出少数类

引用饼干Japson的话：所有真实值为1的数据中，预测对了的个数。每当有100个癌症患者，算法可以成功的预测出8个 。也就是我们关注的那个事件真实的发生情况下，我们成功预测的比例是多少。

### F1 score
 即 F1_measure

因召回率和精确度是此消彼⻓长的，两者之间的平衡代表了捕捉少数类的需求和尽量不要误伤多数类的需求的平衡。究竟要偏向于哪一方，
取决于我们的业务需求：究竟是误伤多数类的成本更高，还是无法捕捉少数类的代价更高。
为了同时兼顾精确度和召回率，我们创造了两者的调和平均数作为考量两者平衡的综合性指标，称之为F1_measure。

F1_measure=2/(1/precision +1/recall)=2*precision*recall/(precision+recall)

为解决样本不平衡问题，我们常用此指标来评估模型的好坏。

## 手工代码实现

将以下代码封装入metrics.py文件中

### TN

In [1]:
def TN(y_true,y_predict):
    assert y_true.shape[0]==y_predict.shape[0],\
        "the size of y_true must be equal to the size of y_predict"
    # (y_true == 0)：向量与数值按位比较，得到的是一个布尔向量
    # 向量与向量按位与，结果还是布尔向量
    # np.sum 计算布尔向量中True的个数(True记为1，False记为0)
    return np.sum((y_true==0)&(y_predict==0)) # 向量与向量按位与，结果还是向量

### FP 

In [4]:
def FP(y_true,y_predict):
    assert y_true.shape[0]==y_predict.shape[0],\
        "the size of y_true must be equal to the size of y_predict"
    # (y_true == 0)：向量与数值按位比较，得到的是一个布尔向量
    # 向量与向量按位与，结果还是布尔向量
    # np.sum 计算布尔向量中True的个数(True记为1，False记为0)
    
    return np.sum((y_true==0)&(y_predict==1)) # 向量与向量按位与，结果还是向量

### FN

In [5]:
def FN(y_true,y_predict):
    assert y_true.shape[0]==y_predict.shape[0],\
        "the size of y_true must be equal to the size of y_predict"
    # (y_true == 1)：向量与数值按位比较，得到的是一个布尔向量
    # 向量与向量按位与，结果还是布尔向量
    # np.sum 计算布尔向量中True的个数(True记为1，False记为0)
    
    return np.sum((y_true==1)&(y_predict==0))  # 向量与向量按位与，结果还是向量

### TP

In [6]:
def TP(y_true,y_predict):
    assert y_true.shape[0]==y_predict.shape[0],\
        "the size of y_true must be equal to the size of y_predict"
    # (y_true == 1)：向量与数值按位比较，得到的是一个布尔向量
    # 向量与向量按位与，结果还是布尔向量
    # np.sum 计算布尔向量中True的个数(True记为1，False记为0)
    
    return np.sum((y_true==1)&(y_predict==1))  # 向量与向量按位与，结果还是向量

### 构建混淆矩阵

In [7]:
def confusion_matrix(y_true,y_predict):
    return np.array([
        [TN(y_true,y_predict),FP(y_true,y_predict)],
        [FN(y_true,y_predict),TP(y_true,y_predict)]
    ])

### 精准率

In [9]:
def precision_score(y_true, y_predict):
    tp=TP(y_true, y_predict)
    fp=FP(y_true, y_predict)
    try:
        return tp/(tp+fp)
    except:
        return 0.0

### 召回率

In [10]:
def recall_score(y_true, y_predict):
    tp=TP(y_true, y_predict)
    fn=FN(y_true, y_predict)
    try:
        return tp/(tp+fn)
    except:
        return 0.0

### F1 score

In [11]:
def f1_measure(y_true, y_predict):
    recall=recall_score(y_true, y_predict)
    precision=precision_score(y_true, y_predict)
    try:
        return 2*recall*precision/(recall+precision)
    except:
        return 0.0

### 小应用

In [2]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

digits = datasets.load_digits()
X = digits.data
y = digits.target.copy()

# 要构造偏斜数据，将数字9的对应索引的元素设置为1，0～8设置为0
y[digits.target==9]=1
y[digits.target!=9]=0

# 使用逻辑回归做一个分类
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)
# 得到X_test所对应的预测值
y_log_predict = log_reg.predict(X_test)

D:\tool\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [3]:
TN(y_test,y_log_predict)

403

In [14]:
FP(y_test,y_log_predict)

2

In [13]:
FN(y_test,y_log_predict)

9

In [12]:
TP(y_test,y_log_predict)

36

In [15]:
precision_score(y_test,y_log_predict)

0.9473684210526315

In [16]:
recall_score(y_test,y_log_predict)

0.8

In [17]:
f1_measure(y_test,y_log_predict)

0.8674698795180723

In [21]:
confusion_matrix(y_test,y_log_predict)

array([[403,   2],
       [  9,  36]], dtype=int64)

## scikit-learn中的混淆矩阵，精准率和召回率

### 混淆矩阵

In [18]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,y_log_predict)

array([[403,   2],
       [  9,  36]], dtype=int64)

### 精准率

In [22]:
from sklearn.metrics import precision_score

precision_score(y_test,y_log_predict)

0.9473684210526315

### 召回率

In [23]:
from sklearn.metrics import recall_score

recall_score(y_test,y_log_predict)

0.8

###  f1_score

In [24]:
from sklearn.metrics import f1_score
f1_score(y_test,y_log_predict)

0.8674698795180723